Week 6 Internt Layer

In [ ]:
import ipaddress 
 
def analyse_ip(ip_str): 
    # Create an IP interface object 
    ip = ipaddress.ip_interface(ip_str) 
     
    print(f"Address: {ip.ip}") 
    print(f"Network: {ip.network}") 
    print(f"Netmask: {ip.netmask}") 
    print(f"Is private: {ip.ip.is_private}") 
    print(f"Is global: {ip.ip.is_global}") 
     
    # List all hosts in the network 
    if ip.network.num_addresses < 256:  # Only for small networks 
        print("\nHosts in network:") 
        for host in ip.network.hosts(): 
            print(host) 
 
# Example usage 
analyse_ip('192.168.1.1')

Exercise 1

In [ ]:
import ipaddress

def analyse_ip(ip_str):
    ip = ipaddress.ip_interface(ip_str)
    
    print(f"Address: {ip.ip}")
    print(f"Network: {ip.network}")
    print(f"Netmask: {ip.netmask}")
    print(f"Private: {ip.ip.is_private}")
    print(f"Global: {ip.ip.is_global}")
    print(f"Broadcast: {ip.network.broadcast_address}")
    
    hosts = list(ip.network.hosts())
    if hosts:
        first_usable_host = hosts[0]
        last_usable_host = hosts[-1]
        print(f"First usable host: {first_usable_host}")
        print(f"Last usable host: {last_usable_host}")
    else:
        print("No usable hosts in this network.")
    
    num_usable_hosts = len(hosts)
    print(f"Number of usable host addresses: {num_usable_hosts}")
    
    if ip.network.num_addresses < 256:
        print("\nHosts in network:")
        for host in hosts:
            print(host)
    
    print("\nComparing networks with different CIDR prefixes:")
    for cidr in range(24, 29):
        new_ip_str = f"{ip.ip}/{cidr}"
        new_ip = ipaddress.ip_interface(new_ip_str)
        print(f"\nNetwork with CIDR /{cidr}:")
        print(f"Network: {new_ip.network}")
        print(f"Netmask: {new_ip.netmask}")
        print(f"Broadcast: {new_ip.network.broadcast_address}")
        hosts = list(new_ip.network.hosts())
        if hosts:
            first_usable_host = hosts[0]
            last_usable_host = hosts[-1]
            print(f"First usable host: {first_usable_host}")
            print(f"Last usable host: {last_usable_host}")
        else:
            print("No usable hosts in this network.")
        num_usable_hosts = len(hosts)
        print(f"Number of usable host addresses: {num_usable_hosts}")

analyse_ip('192.168.1.1/24')

Exercise 2

In [ ]:
import socket
hostname = socket.gethostname()
ipadd = socket.gethostbyname(hostname)
print("Your IP Address is:" + ipadd)

analyse_ip(ipadd)

Exercise 3

In [ ]:
import socket
import ipaddress

def get_ip_address(hostname):
    try:
        ip_address = socket.gethostbyname(hostname)
        return ip_address
    except socket.gaierror:
        print(f"Unable to get IP address for {hostname}")
        return None
    
analyse_ip(get_ip_address('www.gold.ac.uk'))

Exercise 4:

Create a subnetting plan for a company with 4
departments requiring the following hosts:
• Engineering: 30 hosts
• Marketing: 15 hosts
• Finance: 10 hosts
• HR: 5 hosts
Use the network address 172.16.0.0/16 and determine appropriate subnet masks for
each department.

Steps:

1: determine number of hosts per department
2: calculate subnet size
3: assign subnets to each department

Step 1 and 2: 
- each subnet needs to accomodate hosts plus buffer
- we do this by 2x– 2, where x is the number of host ID bits in the IP address

Engineering - 32 hosts, /27 subnet mask
Marketing - 16 hosts, /28 subnet mask
Finance - 16 hosts, /28 subnet mask
HR - 8 hosts, /28 subnet mask

Step 3: assign subnets

Engineering:

Network: 172.16.0.0/27
Usable IP range: 172.16.0.1 - 172.16.0.30
Broadcast: 172.16.0.31

Marketing:

Network: 172.16.0.32/28
Usable IP range: 172.16.0.33 - 172.16.0.46
Broadcast: 172.16.0.47

Finance:

Network: 172.16.0.48/28
Usable IP range: 172.16.0.49 - 172.16.0.62
Broadcast: 172.16.0.63

HR:

Network: 172.16.0.64/29
Usable IP range: 172.16.0.65 - 172.16.0.70
Broadcast: 172.16.0.71

In [ ]:
# Simulating DHCP operations

# dhcp_simple.py - Simplified DHCP Simulator 
 
# Server Configuration 
server = { 
    "ip_pool": ["192.168.1.100", "192.168.1.101", "192.168.1.102"], 
    "leases": {} 
} 
 
# Client Configuration 
client = { 
    "mac": "AA:BB:CC:DD:EE:FF", 
    "ip": None 
} 
 
def send_discover(): 
    print("\n[CLIENT] Step 1: Sending DHCP DISCOVER") 
    return { 
        "type": "DISCOVER", 
        "mac": client["mac"] 
    } 
 
def make_offer(discover): 
    print("\n[SERVER] Step 2: Making DHCP OFFER") 
    if not server["ip_pool"]: 
        print("No IPs available!") 
        return None 
     
    offered_ip = server["ip_pool"].pop(0) 
    return { 
        "type": "OFFER", 
        "mac": discover["mac"], 
        "ip": offered_ip 
    } 
 
def send_request(offer): 
    print("\n[CLIENT] Step 3: Sending DHCP REQUEST") 
    return { 
        "type": "REQUEST", 
        "mac": offer["mac"], 
        "ip": offer["ip"] 
    } 
 
def send_ack(request): 
    print("\n[SERVER] Step 4: Sending DHCP ACK") 
    server["leases"][request["mac"]] = request["ip"] 
    return { 
        "type": "ACK", 
        "mac": request["mac"], 
        "ip": request["ip"] 
    } 

def main(): 
    print("=== Simple DHCP Simulation ===") 
     
    # Client starts process 
    discover = send_discover() 
     
    # Server responds 
    offer = make_offer(discover) 
    if not offer: 
        return 
     
    # Client continues 
    request = send_request(offer) 
     
    # Server finalizes 
    ack = send_ack(request) 
     
    # Update client IP 
    client["ip"] = ack["ip"] 
     
    print("\n=== Result ===") 
    print(f"Client {client['mac']} got IP: {client['ip']}") 
    print("Server leases:", server["leases"]) 
 
if __name__ == "__main__": 
    main()

Exercise 5:

In [ ]:
import socket
import threading

server = {
    "ip_pool": ["192.168.1.100", "192.168.1.101", "192.168.1.102"],
    "leases": {}
}

def handle_client(client_socket):
    while True:
        try:
            data = client_socket.recv(1024).decode()
            if not data:
                break
            
            message = eval(data)
            if message["type"] == "DISCOVER":
                print("\n[SERVER] Making DHCP OFFER")
                if not server["ip_pool"]:
                    print("No IPs available")
                    client_socket.sendall(str({"type": "NACK"}).encode())
                    continue
                
                offered_ip = server["ip_pool"].pop(0)
                offer = {
                    "type": "OFFER",
                    "mac": message["mac"],
                    "ip": offered_ip
                }
                client_socket.sendall(str(offer).encode())
            
            elif message["type"] == "REQUEST":
                print("\n[SERVER] Sending DHCP ACK")
                server["leases"][message["mac"]] = message["ip"]
                ack = {
                    "type": "ACK",
                    "mac": message["mac"],
                    "ip": message["ip"]
                }
                client_socket.sendall(str(ack).encode())
        
        except Exception as e:
            print(f"Error: {e}")
            break
    
    client_socket.close()

def main():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 65432))
    server_socket.listen(5)
    
    print("TCP DHCP Server is waiting")
    
    while True:
        client_socket, client_address = server_socket.accept()
        client_handler = threading.Thread(target=handle_client, args=(client_socket,))
        client_handler.start()

if __name__ == "__main__":
    main()

In [ ]:
import socket

client = {
    "mac": "AA:BB:CC:DD:EE:FF",
    "ip": None
}

def send_discover(client_socket):
    print("\n[CLIENT] Step 1: Sending DHCP DISCOVER")
    discover = {
        "type": "DISCOVER",
        "mac": client["mac"]
    }
    client_socket.sendall(str(discover).encode())

def send_request(client_socket, offer):
    print("\n[CLIENT] Step 3: Sending DHCP REQUEST")
    request = {
        "type": "REQUEST",
        "mac": offer["mac"],
        "ip": offer["ip"]
    }
    client_socket.sendall(str(request).encode())

def main():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('localhost', 65432))
    
    send_discover(client_socket)
    
    offer_data = client_socket.recv(1024).decode()
    offer = eval(offer_data)
    if offer["type"] == "NACK":
        print("No IPs available")
        client_socket.close()
        return
    
    print(f"\n[CLIENT] Received DHCP: {offer}")
    
    send_request(client_socket, offer)
    
    ack_data = client_socket.recv(1024).decode()
    ack = eval(ack_data)
    print(f"\n[CLIENT] Received ACK: {ack}")
    
    client["ip"] = ack["ip"]
    
    print("\n=== Result ===")
    print(f"Client {client['mac']} got IP: {client['ip']}")
    
    client_socket.close()

if __name__ == "__main__":
    main()